### T5 for text simplification v1

In [ ]:
!pip install transformers -q
!pip install datasets -q
!pip install evaluate -q
!pip install sentencepiece -q
!pip install accelerate -q
!pip install rouge_score -q
!pip install sacrebleu -q
!pip install sacremoses -q
!pip install py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import pipeline, set_seed
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datasets import load_dataset, load_metric
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, T5ForConditionalGeneration
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
import torch
from tqdm import tqdm

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
dataset = load_dataset("bogdancazan/wikilarge-text-simplification")
dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/bogdancazan___csv/bogdancazan--wikilarge-text-simplification-6bc5552af2c5d584/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Normal', 'Simple'],
        num_rows: 148843
    })
    validation: Dataset({
        features: ['Normal', 'Simple'],
        num_rows: 494
    })
    test: Dataset({
        features: ['Normal', 'Simple'],
        num_rows: 191
    })
})

In [ ]:
bleu = load_metric('bleu')
rouge = load_metric('rouge')
sari = load_metric('sari')

<ipython-input-6-77d0ad785063>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu = load_metric('bleu')


In [ ]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements.

    Generator function to yield successive batch-sized chunks from list_of_elements.

    Parameters:
    list_of_elements (list): List of elements to be divided into chunks.
    batch_size (int): The size of each chunk.

    Yields:
    list: Batch-sized chunk from list_of_elements.

    """
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    """
    Function to calculate a specified metric on a test dataset for a Natural Language Processing (NLP) task.
    It assumes the task is a text summarization task, where the goal is to generate a summary from a text.

    Parameters:
    dataset (pandas.DataFrame): The test dataset. It should contain a column for the text and a column for the true summary.
    metric (datasets.Metric): The metric to calculate. This should be a metric object from the Hugging Face datasets library.
    model (transformers.PreTrainedModel): The transformer model to use for text generation.
    tokenizer (transformers.PreTrainedTokenizer): The tokenizer corresponding to the model.
    batch_size (int, optional): The size of the batches to use for processing. Defaults to 16.
    device (str, optional): The device to run the model on. Defaults to the output of torch.cuda.is_available().
    column_text (str, optional): The name of the column in the dataset that contains the text. Defaults to 'article'.
    column_summary (str, optional): The name of the column in the dataset that contains the true summary. Defaults to 'highlights'.

    Returns:
    score (float): The calculated score of the metric on the test dataset.
    """
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the <n> token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

In [ ]:
model_name = 'bogdancazan/t5-small-wikilarge-text-simplification-penalty-loss'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_t5 = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

In [ ]:
def convert_examples_to_features_hf(example_batch):
  # example_batch['Normal'] = ['simplify: ' + line for line in example_batch['Normal']]
  input_encodings = tokenizer(example_batch['Normal'], max_length=80, truncation=True, padding=True, return_tensors='pt')

  target_encodings = tokenizer(example_batch['Simple'], max_length=80, truncation=True, padding=True, return_tensors='pt')

  return {
      'input_ids' : input_encodings['input_ids'],
      'attention_mask': input_encodings['attention_mask'],
      'labels': target_encodings['input_ids'],
  }

In [ ]:
dataset_wikilarge_pt = dataset.map(convert_examples_to_features_hf, batched=True)

Map:   0%|          | 0/148843 [00:00<?, ? examples/s]

Map:   0%|          | 0/494 [00:00<?, ? examples/s]

Map:   0%|          | 0/191 [00:00<?, ? examples/s]

In [ ]:
dataset_wikilarge_pt

DatasetDict({
    train: Dataset({
        features: ['Normal', 'Simple', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 148843
    })
    validation: Dataset({
        features: ['Normal', 'Simple', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 494
    })
    test: Dataset({
        features: ['Normal', 'Simple', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 191
    })
})

In [ ]:
from transformers import DataCollatorForSeq2Seq, TrainingArguments, Trainer, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_t5)


In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    rouge_metric = load_metric('rouge')

    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
def compute_bleu_score(references, predictions):
    total_score = 0.0
    num_pairs = len(references)

    for reference, prediction in zip(references, predictions):
        reference_tokens = nltk.word_tokenize(reference)
        prediction_tokens = nltk.word_tokenize(prediction)

        smoothing_function = nltk.translate.bleu_score.SmoothingFunction()
        bleu_score = nltk.translate.bleu_score.sentence_bleu([reference_tokens], prediction_tokens, smoothing_function=smoothing_function.method7)

        total_score += bleu_score

    average_bleu_score = total_score / num_pairs
    return average_bleu_score

In [ ]:
class CustomTrainer(Trainer):

  def __init__(self,
        model=None,
        args=None,
        data_collator=None,
        train_dataset=None,
        eval_dataset=None,
        tokenizer=None,
        model_init=None,
        compute_metrics=None,
        callbacks=None,
        optimizers=(None, None),
        preprocess_logits_for_metrics=None,
      ) -> None:
      super().__init__(model=model, args=args, data_collator=data_collator, train_dataset=train_dataset, eval_dataset=eval_dataset, tokenizer=tokenizer, model_init=model_init, compute_metrics=compute_metrics, callbacks=callbacks, optimizers=optimizers, preprocess_logits_for_metrics=preprocess_logits_for_metrics)
      self.iteration = 0
      self.alpha = 0.6


  def compute_loss(self, model, inputs, return_outputs=False):
    """
    How the loss is computed by Trainer. By default, all models return the loss in the first element.

    Subclass and override for custom behavior.
    """
    if self.label_smoother is not None and "labels" in inputs:
            labels = inputs.pop("labels")
    else:
        labels = None
    outputs = model(**inputs)
    # Save past state if it exists
    # TODO: this needs to be fixed and made cleaner later.
    if self.args.past_index >= 0:
        self._past = outputs[self.args.past_index]

    if labels is not None:
        if unwrap_model(model)._get_name() in MODEL_FOR_CAUSAL_LM_MAPPING_NAMES.values():
            loss = self.label_smoother(outputs, labels, shift_labels=True)
        else:
            loss = self.label_smoother(outputs, labels)
    else:
        if isinstance(outputs, dict) and "loss" not in outputs:
            raise ValueError(
                "The model did not return a loss from the inputs, only the following keys: "
                f"{','.join(outputs.keys())}. For reference, the inputs it received are {','.join(inputs.keys())}."
            )
        # We don't use .loss here since the model may return tuples instead of ModelOutput.
        loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]
        self.iteration += 1
        if self.iteration % 32 == 0:
          self.alpha *= 0.9

        outputs = model(**inputs)


        my_ids = inputs['input_ids']
        my_mask = inputs['attention_mask']
        my_labels = [torch.where(labels != -100, labels, tokenizer.pad_token_id) for labels in inputs['labels']]
        references = [tokenizer.decode(label, skip_special_tokens=True, clean_up_tokenization_spaces=True) for label in my_labels]

        generated_ids = model_t5.generate(
          input_ids=my_ids,
          attention_mask=my_mask,
          max_length=128,
          num_beams=2,
          repetition_penalty=2.5,
          length_penalty=1.0,
          early_stopping=True
        )
        predictions = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]

        loss = outputs["loss"] + self.alpha * compute_bleu_score(references=references, predictions=predictions)

    return (loss, outputs) if return_outputs else loss

In [ ]:
training_args = TrainingArguments(
    output_dir='t5-small-wikilarge-text-simplification-penalty-loss',
    num_train_epochs=4,
    warmup_steps=250,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    logging_steps=10,
    evaluation_strategy='epoch',
    learning_rate=2e-4,
    eval_steps=500,
    save_steps=1e6,
    # gradient_accumulation_steps=16,
    fp16=True,
    optim="adafactor",
    push_to_hub=True,
)

In [ ]:
trainer = CustomTrainer(
    model=model_t5,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=seq2seq_data_collator,
    train_dataset=dataset_wikilarge_pt["train"],
    eval_dataset=dataset_wikilarge_pt["validation"]
)

/content/t5-small-wikilarge-text-simplification-penalty-loss is already a clone of https://huggingface.co/bogdancazan/t5-small-wikilarge-text-simplification-penalty-loss. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.516100,0.499712
2,0.476400,0.495625
3,0.462100,0.493505
4,0.482600,0.492955


TrainOutput(global_step=18608, training_loss=0.4824583969516992, metrics={'train_runtime': 27725.5631, 'train_samples_per_second': 21.474, 'train_steps_per_second': 0.671, 'total_flos': 1.259042484781056e+16, 'train_loss': 0.4824583969516992, 'epoch': 4.0})

In [ ]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/231M [00:00<?, ?B/s]

Upload file training_args.bin:   0%|          | 1.00/3.87k [00:00<?, ?B/s]

Upload file runs/Jun11_22-09-42_284834d27216/events.out.tfevents.1686521386.284834d27216.636.1:   0%|         …

Upload file runs/Jun11_22-08-16_284834d27216/events.out.tfevents.1686521355.284834d27216.636.0:   0%|         …

To https://huggingface.co/bogdancazan/t5-small-wikilarge-text-simplification-penalty-loss
   e278a83..050e9e6  main -> main

   e278a83..050e9e6  main -> main

To https://huggingface.co/bogdancazan/t5-small-wikilarge-text-simplification-penalty-loss
   050e9e6..1a0fc01  main -> main

   050e9e6..1a0fc01  main -> main



'https://huggingface.co/bogdancazan/t5-small-wikilarge-text-simplification-penalty-loss/commit/050e9e66c030ee1db99587b06ed5d198b29f7b18'

In [ ]:
text = "the bushehr nuclear power plant lrb rrb is a nuclear power plant in iran south east of the city of bushehr between the fishing villages of halileh and bandargeh along the persian gulf."

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")
inputs = tokenizer(text, return_tensors="pt").input_ids

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

In [ ]:
outputs = model.generate(inputs, max_new_tokens=2*len(text), do_sample=False)
tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
rouge_names = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']

rouge_metric = load_metric('rouge')

score = calculate_metric_on_test_ds(
    dataset['test'], rouge_metric, trainer.model, tokenizer, batch_size=8, column_text='Normal', column_summary='Simple'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)

pd.DataFrame(rouge_dict, index=[f't5_small'])

In [ ]:
# model_ckpt = "google/pegasus-cnn_dailymail"

# tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

# model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

# score = calculate_metric_on_test_ds(test_sampled, rouge_metric,
#                                    model_pegasus, tokenizer, batch_size=8)

# rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)

# # At the end, we compute and return the ROUGE scores.
# pd.DataFrame(rouge_dict, index=["pegasus"])